In [202]:
def onepl_lsrm_cont_missing(data,

                            ndim,
                            niter,
                            nburn,
                            nthin,
                            nprint,

                            jump_beta,
                            jump_theta,
                            jump_gamma,
                            jump_z,
                            jump_w,

                            pr_mean_beta,
                            pr_sd_beta,
                            pr_a_th_sigma,
                            pr_b_th_sigma,
                            pr_mean_theta,

                            pr_a_sigma,
                            pr_b_sigma,
                            pr_mean_gamma,
                            pr_sd_gamma,

                            missing):
    ##############################################################################

    def update_function(beta, theta, gamma):
        return (- pow((data[k, i] - beta[i] - theta[k] + gamma * dist[k, i]), 2) /
                (2 * pow(pr_sd, 2))
                )

    def rejection_algorithm(ratio):
        if ratio > 0.0:
            accept = 1
        else:
            un = np.random.uniform(1)
            if np.log(un) < ratio:
                accept = 1
            else:
                accept = 0

        return accept

    ##############################################################################

    nsample = data.shape[0]
    nitem = data.shape[1]

    pr_mean_z = pr_mean_w = 0.0
    pr_sd_z = pr_sd_w = pr_sd = pr_sd_theta = 1.0

    oldbeta = np.random.uniform(size=nitem)
    oldtheta = np.random.uniform(size=nsample)
    oldz = np.random.uniform(size=(nsample, ndim))
    oldw = np.random.uniform(size=(nitem, ndim))

    # oldbeta = np.random.uniform(nitem)
    # oldtheta = np.random.uniform(nsample)
    # oldz = np.random.rand(nsample, ndim)
    # oldw = np.random.rand(nitem, ndim)

    newbeta = oldbeta = oldbeta * 4.0 - 2.0
    newtheta = oldtheta = oldtheta * 4.0 - 2.0
    newz = oldz = oldz * 2.0 - 1.0
    neww = oldw = oldw * 2.0 - 1.0

    ##############################################################################

    oldgamma = newgamma = 1  # gamma1 = log(gamma)

    samp_beta = np.zeros(shape=((niter - nburn) // nthin, nitem))
    samp_theta = np.zeros(shape=((niter - nburn) // nthin, nsample))

    samp_z = np.zeros(shape=((niter - nburn) // nthin, nsample, ndim))
    samp_w = np.zeros(shape=((niter - nburn) // nthin, nitem, ndim))

    samp_sd_theta = np.zeros(shape=(niter - nburn) // nthin, )
    samp_sd = np.zeros(shape=(niter - nburn) // nthin, )
    samp_mle = np.zeros(shape=(niter - nburn) // nthin, )
    samp_gamma = np.zeros(shape=(niter - nburn) // nthin, )

    accept_beta = np.zeros(shape=nitem, )
    accept_theta = np.zeros(shape=nsample, )
    accept_z = np.zeros(shape=nsample, )
    accept_w = np.zeros(shape=nitem, )

    accept_gamma = 0
    accept = 0
    count = 0

    dist = np.zeros(shape=(nsample, nitem), )

    old_dist_k = np.zeros(nitem, )
    new_dist_k = np.zeros(nitem, )
    old_dist_i = np.zeros(nsample, )
    new_dist_i = np.zeros(nsample, )

    ##############################################################################

    for iter in range(niter):
        # dist(j,i) is distance of z_j and w_i

        dist = np.where(True, 0, dist)  # 매 이터레이션마다 거리 매트릭스를 0으로 리셋

        for i in range(nitem):
            for k in range(nsample):
                dist_temp = 0.0
                for j in range(ndim):
                    dist_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                    dist[k, i] = sqrt(dist_temp)

        # beta update
        for i in range(nitem):
            # TODO 컬럼부터 갱신하는 이유가 있는건가?

            newbeta[i] = np.random.normal(oldbeta[i], jump_beta, 1)
            old_like_beta = new_like_beta = 0.0

            for k in range(nsample):
                if data[k, i] != missing:
                    new_like_beta += update_function(newbeta, oldtheta, oldgamma)
                    old_like_beta += update_function(oldbeta, oldtheta, oldgamma)

            num = (new_like_beta +
                   scipy.stats.norm.logpdf(newbeta[i], pr_mean_beta, pr_sd_beta))
            den = (old_like_beta +
                   scipy.stats.norm.logpdf(oldbeta[i], pr_mean_beta, pr_sd_beta))
            ratio = num - den

            accept = rejection_algorithm(ratio)


            if accept == 1:
                oldbeta[i] = newbeta[i]
                accept_beta[i] += 1.0 / (niter * 1.0)

            else:
                newbeta[i] = oldbeta[i]

        # theta update
        for k in range(nsample):
            newtheta[k] = np.random.normal(oldtheta[k], jump_theta, 1)
            old_like_theta = new_like_theta = 0.0

            for i in range(nitem):
                if data[k, i] != missing:
                    new_like_theta += update_function(oldbeta, newtheta, oldgamma)
                    old_like_theta += update_function(oldbeta, oldtheta, oldgamma)

            num = (new_like_theta +
                   scipy.stats.norm.logpdf(newtheta[k], pr_mean_theta, pr_sd_theta))
            den = (old_like_theta +
                   scipy.stats.norm.logpdf(oldtheta[k], pr_mean_theta, pr_sd_theta))
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                oldtheta[k] = newtheta[k]
                accept_theta[k] += 1.0 / (niter * 1.0)
            else:
                newtheta[k] = oldtheta[k]

        # gamma(log(gamma)) update

        newgamma = np.random.lognormal(log(oldgamma), jump_gamma, 1)
        old_like_gamma = new_like_gamma = 0.0

        for k in range(nsample):
            for i in range(nitem):
                if data[k, i] != missing:
                    new_like_gamma += update_function(oldbeta, newtheta, newgamma)
                    old_like_gamma += update_function(oldbeta, newtheta, oldgamma)

        num = (new_like_gamma +
               scipy.stats.lognorm.logpdf(oldgamma, s=jump_gamma, loc=0, scale=exp(log(newgamma))) +
               scipy.stats.lognorm.logpdf(newgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
               )
        den = (old_like_gamma +
               scipy.stats.lognorm.logpdf(newgamma, s=jump_gamma, loc=0, scale=exp(log(oldgamma))) +
               scipy.stats.lognorm.logpdf(oldgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
               )
        ratio = num - den

        accept = rejection_algorithm(ratio)

        if accept == 1:
            oldgamma = newgamma
            accept_gamma += 1.0 / (niter * 1.0)
        else:
            newgamma = oldgamma

        # zj update

        for k in range(nsample):
            for j in range(ndim):
                newz[k, j] = np.random.normal(oldz[k, j], jump_z, 1)
            old_like_z = new_like_z = 0.0

            # calculate distance of oldw and newz

            for i in range(nitem):
                dist_old_temp = dist_new_temp = 0.0
                for j in range(ndim):
                    dist_new_temp += pow((newz[k, j] - oldw[i, j]), 2.0)
                    dist_old_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                new_dist_k[i] = sqrt(dist_new_temp)
                old_dist_k[i] = sqrt(dist_old_temp)

            # calculate likelihood

            for i in range(nitem):
                if data[k, i] != missing:
                    new_like_z += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * new_dist_k[i]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )
                    old_like_z += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * old_dist_k[i]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )

            num = den = 0.0

            for j in range(ndim):
                num += scipy.stats.norm.logpdf(newz[k, j], pr_mean_z, pr_sd_z)
                den += scipy.stats.norm.logpdf(oldz[k, j], pr_mean_z, pr_sd_z)

            # Rprintf("%.3f %.3f %.3f %.3f\n", num, den, new_like_z, old_like_z)
            # arma::dvec newzz = dmvnorm(newz.cols(2*j,2*j+1),pr_mean_z,pr_cov_z,TRUE)
            # arma::dvec oldzz = dmvnorm(oldz.cols(2*j,2*j+1),pr_mean_z,pr_cov_z,TRUE)

            num += new_like_z
            den += old_like_z
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                for j in range(ndim):
                    oldz[k, j] = newz[k, j]
                accept_z[k] += 1.0 / (niter * 1.0)

            else:
                for j in range(ndim):
                    newz[k, j] = oldz[k, j]

                #  wi update
        for i in range(nitem):
            for j in range(ndim):
                neww[i, j] = np.random.normal(oldw[i, j], jump_w, 1)
            old_like_w = new_like_w = 0.0

            # calculate distance of neww and oldz

            for k in range(nsample):
                dist_old_temp = dist_new_temp = 0.0
                for j in range(ndim):
                    dist_new_temp += pow((oldz[k, j] - neww[i, j]), 2.0)  # TODO: Why Old - New?
                    dist_old_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                new_dist_i[k] = sqrt(dist_new_temp)
                old_dist_i[k] = sqrt(dist_old_temp)

            # calculate likelihood

            for k in range(nsample):
                if data[k, i] != missing:
                    new_like_w += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * new_dist_i[k]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )
                    old_like_w += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * old_dist_i[k]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )

            num = den = 0.0

            for j in range(ndim):
                num += scipy.stats.norm.logpdf(neww[i, j], pr_mean_w, pr_sd_w)
                den += scipy.stats.norm.logpdf(oldw[i, j], pr_mean_w, pr_sd_w)

            num += new_like_w
            den += old_like_w
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                for j in range(ndim):
                    oldw[i, j] = neww[i, j]
                accept_w[i] += 1.0 / (niter * 1.0)
            else:
                for j in range(ndim):
                    neww[i, j] = oldw[i, j]

        # sigma_theta update with gibbs

        post_a_th_sigma = pr_a_th_sigma * 2 + nsample
        post_b_th_sigma = pr_b_th_sigma

        for j in range(nsample):  # TODO: 여기 j인데 nsample인거 맞음?
            post_b_th_sigma += pow((oldtheta[j] - pr_mean_theta), 2.0)
        pr_sd_theta = sqrt(2 * post_b_th_sigma * (1.0 / np.random.chisquare(post_a_th_sigma)))

        # dist(j,i) is distance of z_j and w_i

        dist = np.where(True, 0, dist)

        for i in range(nitem):
            for k in range(nsample):
                dist_temp = 0.0
                for j in range(ndim):
                    dist_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                dist[k, i] = sqrt(dist_temp)

        # sigma update with gibbs

        post_a_sigma = pr_a_sigma * 2 + nsample * nitem
        post_b_sigma = pr_b_sigma

        for j in range(nsample):  # TODO: 여기 j인데 nsample인거 맞음?
            for i in range(nitem):
                post_b_sigma += pow((data[j, i] - oldbeta[i] - oldtheta[j] + oldgamma * dist[j, i]), 2.0) / 2

        pr_sd = sqrt(2 * post_b_sigma * (1.0 / np.random.chisquare(post_a_sigma)))

        # burn, thin

        if iter >= nburn and iter % nthin == 0:
            for i in range(nitem):
                samp_beta[count, i] = oldbeta[i]
            for k in range(nsample):
                samp_theta[count, k] = oldtheta[k]
            for i in range(nitem):
                for j in range(ndim):
                    samp_w[count, i, j] = oldw[i, j]
            for k in range(nsample):
                for j in range(ndim):
                    samp_z[count, k, j] = oldz[k, j]

            samp_gamma[count] = oldgamma
            samp_sd_theta[count] = pr_sd_theta
            samp_sd[count] = pr_sd

            mle = 0.0

            for i in range(nitem):
                mle += scipy.stats.norm.logpdf(oldbeta[i], pr_mean_beta, pr_sd_beta)
            for k in range(nsample):
                mle += scipy.stats.norm.logpdf(oldtheta[k], pr_mean_theta, pr_sd_theta)
            for i in range(nitem):
                for j in range(ndim):
                    mle += scipy.stats.norm.logpdf(oldw[i, j], pr_mean_w, pr_sd_w)
            for k in range(nsample):
                for j in range(ndim):
                    mle += scipy.stats.norm.logpdf(oldz[k, j], pr_mean_z, pr_sd_z)
            for k in range(nsample):
                for i in range(nitem):
                    mle += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * dist[k, i]), 2)
                            / (2 * pow(pr_sd, 2)))

            mle += scipy.stats.lognorm.logpdf(oldgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
            samp_mle[count] = mle

            count += 1

        if iter % nprint == 0:
            print("Iteration: ", iter)
            print("count", count)
            for i in range(nitem):
                print("nitem: ", i, " with ", oldbeta[i])
            print("oldgamma: ", oldgamma, "     pr_sd_theta: ", pr_sd_theta)

    return {"beta": samp_beta,
            "theta": samp_theta,
            "z": samp_z,
            "w": samp_w,
            "gamma": samp_gamma,
            "sigma_theta": samp_sd_theta,
            "sigma": samp_sd,
            "map": samp_mle,
            "accept_beta": accept_beta,
            "accept_theta": accept_theta,
            "accept_z": accept_z,
            "accept_w": accept_w,
            "accept_gamma": accept_gamma}








In [224]:
data = logit_x

In [219]:
##############################################################################

def update_function(beta, theta, gamma):
    return (- pow((data[k, i] - beta[i] - theta[k] + gamma * dist[k, i]), 2) /
            (2 * pow(pr_sd, 2))
            )

def rejection_algorithm(ratio):
    if ratio > 0.0:
        accept = 1
    else:
        un = np.random.uniform(1)
        if np.log(un) < ratio:
            accept = 1
        else:
            accept = 0

    return accept

##############################################################################

nsample = data.shape[0]
nitem = data.shape[1]

pr_mean_z = pr_mean_w = 0.0
pr_sd_z = pr_sd_w = pr_sd = pr_sd_theta = 1.0

oldbeta = np.random.uniform(size=nitem)
oldtheta = np.random.uniform(size=nsample)
oldz = np.random.uniform(size=(nsample, ndim))
oldw = np.random.uniform(size=(nitem, ndim))

# oldbeta = np.random.uniform(nitem)
# oldtheta = np.random.uniform(nsample)
# oldz = np.random.rand(nsample, ndim)
# oldw = np.random.rand(nitem, ndim)

oldbeta = oldbeta * 4.0 - 2.0
oldtheta = oldtheta * 4.0 - 2.0
oldz = oldz * 2.0 - 1.0
oldw = oldw * 2.0 - 1.0

newbeta = oldbeta 
newtheta = oldtheta
newz = oldz 
neww = oldw 

##############################################################################

oldgamma = 1
newgamma = 1  # gamma1 = log(gamma)

samp_beta = np.zeros(shape=((niter - nburn) // nthin, nitem))
samp_theta = np.zeros(shape=((niter - nburn) // nthin, nsample))

samp_z = np.zeros(shape=((niter - nburn) // nthin, nsample, ndim))
samp_w = np.zeros(shape=((niter - nburn) // nthin, nitem, ndim))

samp_sd_theta = np.zeros(shape=(niter - nburn) // nthin, )
samp_sd = np.zeros(shape=(niter - nburn) // nthin, )
samp_mle = np.zeros(shape=(niter - nburn) // nthin, )
samp_gamma = np.zeros(shape=(niter - nburn) // nthin, )

accept_beta = np.zeros(shape=nitem, )
accept_theta = np.zeros(shape=nsample, )
accept_z = np.zeros(shape=nsample, )
accept_w = np.zeros(shape=nitem, )

accept_gamma = 0
accept = 0
count = 0

dist = np.zeros(shape=(nsample, nitem), )

old_dist_k = np.zeros(nitem, )
new_dist_k = np.zeros(nitem, )
old_dist_i = np.zeros(nsample, )
new_dist_i = np.zeros(nsample, )

##############################################################################

for iter in range(niter):
    # dist(j,i) is distance of z_j and w_i

    dist = np.where(True, 0, dist)  # 매 이터레이션마다 거리 매트릭스를 0으로 리셋

    for i in range(nitem):
        for k in range(nsample):
            dist_temp = 0.0
            for j in range(ndim):
                dist_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                dist[k, i] = sqrt(dist_temp)

    # beta update
    for i in range(nitem):
        # TODO 컬럼부터 갱신하는 이유가 있는건가?

        newbeta[i] = np.random.normal(oldbeta[i], jump_beta, 1)
        old_like_beta = new_like_beta = 0.0

        for k in range(nsample):
            if data[k, i] != missing:
                new_like_beta += update_function(newbeta, oldtheta, oldgamma)
                old_like_beta += update_function(oldbeta, oldtheta, oldgamma)

        num = (new_like_beta +
               scipy.stats.norm.logpdf(newbeta[i], pr_mean_beta, pr_sd_beta))
        den = (old_like_beta +
               scipy.stats.norm.logpdf(oldbeta[i], pr_mean_beta, pr_sd_beta))
        ratio = num - den

        accept = rejection_algorithm(ratio)


        if accept == 1:
            oldbeta[i] = newbeta[i]
            accept_beta[i] += 1.0 / (niter * 1.0)

        else:
            newbeta[i] = oldbeta[i]

    # theta update
    for k in range(nsample):
        newtheta[k] = np.random.normal(oldtheta[k], jump_theta, 1)
        old_like_theta = new_like_theta = 0.0

        for i in range(nitem):
            if data[k, i] != missing:
                new_like_theta += update_function(oldbeta, newtheta, oldgamma)
                old_like_theta += update_function(oldbeta, oldtheta, oldgamma)

        num = (new_like_theta +
               scipy.stats.norm.logpdf(newtheta[k], pr_mean_theta, pr_sd_theta))
        den = (old_like_theta +
               scipy.stats.norm.logpdf(oldtheta[k], pr_mean_theta, pr_sd_theta))
        ratio = num - den

        accept = rejection_algorithm(ratio)

        if accept == 1:
            oldtheta[k] = newtheta[k]
            accept_theta[k] += 1.0 / (niter * 1.0)
        else:
            newtheta[k] = oldtheta[k]

    # gamma(log(gamma)) update

    newgamma = np.random.lognormal(log(oldgamma), jump_gamma, 1)
    old_like_gamma = new_like_gamma = 0.0

    for k in range(nsample):
        for i in range(nitem):
            if data[k, i] != missing:
                new_like_gamma += update_function(oldbeta, newtheta, newgamma)
                old_like_gamma += update_function(oldbeta, newtheta, oldgamma)

    num = (new_like_gamma +
           scipy.stats.lognorm.logpdf(oldgamma, s=jump_gamma, loc=0, scale=exp(log(newgamma))) +
           scipy.stats.lognorm.logpdf(newgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
           )
    den = (old_like_gamma +
           scipy.stats.lognorm.logpdf(newgamma, s=jump_gamma, loc=0, scale=exp(log(oldgamma))) +
           scipy.stats.lognorm.logpdf(oldgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
           )
    ratio = num - den

    accept = rejection_algorithm(ratio)

    if accept == 1:
        oldgamma = newgamma
        accept_gamma += 1.0 / (niter * 1.0)
    else:
        newgamma = oldgamma

    # zj update

    for k in range(nsample):
        for j in range(ndim):
            newz[k, j] = np.random.normal(oldz[k, j], jump_z, 1)
        old_like_z = new_like_z = 0.0

        # calculate distance of oldw and newz

        for i in range(nitem):
            dist_old_temp = dist_new_temp = 0.0
            
            for j in range(ndim):
                dist_new_temp += pow((newz[k, j] - oldw[i, j]), 2.0)
                dist_old_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
            new_dist_k[i] = sqrt(dist_new_temp)
            old_dist_k[i] = sqrt(dist_old_temp)

        # calculate likelihood

        for i in range(nitem):
            if data[k, i] != missing:
                new_like_z += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * new_dist_k[i]), 2) /
                               (2 * pow(pr_sd, 2))
                               )
                old_like_z += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * old_dist_k[i]), 2) /
                               (2 * pow(pr_sd, 2))
                               )

        num = den = 0.0

        for j in range(ndim):
            num += scipy.stats.norm.logpdf(newz[k, j], pr_mean_z, pr_sd_z)
            den += scipy.stats.norm.logpdf(oldz[k, j], pr_mean_z, pr_sd_z)

        # Rprintf("%.3f %.3f %.3f %.3f\n", num, den, new_like_z, old_like_z)
        # arma::dvec newzz = dmvnorm(newz.cols(2*j,2*j+1),pr_mean_z,pr_cov_z,TRUE)
        # arma::dvec oldzz = dmvnorm(oldz.cols(2*j,2*j+1),pr_mean_z,pr_cov_z,TRUE)

        num += new_like_z
        den += old_like_z
        ratio = num - den

        accept = rejection_algorithm(ratio)

        if accept == 1:
            for j in range(ndim):
                oldz[k, j] = newz[k, j]
            accept_z[k] += 1.0 / (niter * 1.0)

        else:
            for j in range(ndim):
                newz[k, j] = oldz[k, j]

            #  wi update
    for i in range(nitem):
        for j in range(ndim):
            neww[i, j] = np.random.normal(oldw[i, j], jump_w, 1)
        old_like_w = new_like_w = 0.0

        # calculate distance of neww and oldz

        for k in range(nsample):
            dist_old_temp = dist_new_temp = 0.0
            for j in range(ndim):
                dist_new_temp += pow((oldz[k, j] - neww[i, j]), 2.0)  # TODO: Why Old - New?
                dist_old_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
            new_dist_i[k] = sqrt(dist_new_temp)
            old_dist_i[k] = sqrt(dist_old_temp)

        # calculate likelihood

        for k in range(nsample):
            if data[k, i] != missing:
                new_like_w += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * new_dist_i[k]), 2) /
                               (2 * pow(pr_sd, 2))
                               )
                old_like_w += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * old_dist_i[k]), 2) /
                               (2 * pow(pr_sd, 2))
                               )

        num = den = 0.0

        for j in range(ndim):
            num += scipy.stats.norm.logpdf(neww[i, j], pr_mean_w, pr_sd_w)
            den += scipy.stats.norm.logpdf(oldw[i, j], pr_mean_w, pr_sd_w)

        num += new_like_w
        den += old_like_w
        ratio = num - den

        accept = rejection_algorithm(ratio)

        if accept == 1:
            for j in range(ndim):
                oldw[i, j] = neww[i, j]
            accept_w[i] += 1.0 / (niter * 1.0)
        else:
            for j in range(ndim):
                neww[i, j] = oldw[i, j]

    # sigma_theta update with gibbs

    post_a_th_sigma = pr_a_th_sigma * 2 + nsample
    post_b_th_sigma = pr_b_th_sigma

    for j in range(nsample):  # TODO: 여기 j인데 nsample인거 맞음?
        post_b_th_sigma += pow((oldtheta[j] - pr_mean_theta), 2.0)
    pr_sd_theta = sqrt(2 * post_b_th_sigma * (1.0 / np.random.chisquare(post_a_th_sigma)))

    # dist(j,i) is distance of z_j and w_i

    dist = np.where(True, 0, dist)

    for i in range(nitem):
        for k in range(nsample):
            dist_temp = 0.0
            for j in range(ndim):
                dist_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
            dist[k, i] = sqrt(dist_temp)

    # sigma update with gibbs

    post_a_sigma = pr_a_sigma * 2 + nsample * nitem
    post_b_sigma = pr_b_sigma

    for j in range(nsample):  # TODO: 여기 j인데 nsample인거 맞음?
        for i in range(nitem):
            post_b_sigma += pow((data[j, i] - oldbeta[i] - oldtheta[j] + oldgamma * dist[j, i]), 2.0) / 2

    pr_sd = sqrt(2 * post_b_sigma * (1.0 / np.random.chisquare(post_a_sigma)))

    # burn, thin

    if iter >= nburn and iter % nthin == 0:
        for i in range(nitem):
            samp_beta[count, i] = oldbeta[i]
        for k in range(nsample):
            samp_theta[count, k] = oldtheta[k]
        for i in range(nitem):
            for j in range(ndim):
                samp_w[count, i, j] = oldw[i, j]
        for k in range(nsample):
            for j in range(ndim):
                samp_z[count, k, j] = oldz[k, j]

        samp_gamma[count] = oldgamma
        samp_sd_theta[count] = pr_sd_theta
        samp_sd[count] = pr_sd

        mle = 0.0

        for i in range(nitem):
            mle += scipy.stats.norm.logpdf(oldbeta[i], pr_mean_beta, pr_sd_beta)
        for k in range(nsample):
            mle += scipy.stats.norm.logpdf(oldtheta[k], pr_mean_theta, pr_sd_theta)
        for i in range(nitem):
            for j in range(ndim):
                mle += scipy.stats.norm.logpdf(oldw[i, j], pr_mean_w, pr_sd_w)
        for k in range(nsample):
            for j in range(ndim):
                mle += scipy.stats.norm.logpdf(oldz[k, j], pr_mean_z, pr_sd_z)
        for k in range(nsample):
            for i in range(nitem):
                mle += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * dist[k, i]), 2)
                        / (2 * pow(pr_sd, 2)))

        mle += scipy.stats.lognorm.logpdf(oldgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
        samp_mle[count] = mle

        count += 1

    if iter % nprint == 0:
        print("Iteration: ", iter)
        print("count", count)
        for i in range(nitem):
            print("nitem: ", i, " with ", oldbeta[i])
        print("oldgamma: ", oldgamma, "     pr_sd_theta: ", pr_sd_theta)

return {"beta": samp_beta,
        "theta": samp_theta,
        "z": samp_z,
        "w": samp_w,
        "gamma": samp_gamma,
        "sigma_theta": samp_sd_theta,
        "sigma": samp_sd,
        "map": samp_mle,
        "accept_beta": accept_beta,
        "accept_theta": accept_theta,
        "accept_z": accept_z,
        "accept_w": accept_w,
        "accept_gamma": accept_gamma}








<ipython-input-219-db07af29f152>:104: RuntimeWarning: invalid value encountered in double_scalars
  ratio = num - den
<ipython-input-219-db07af29f152>:130: RuntimeWarning: invalid value encountered in double_scalars
  ratio = num - den
<ipython-input-219-db07af29f152>:159: RuntimeWarning: invalid value encountered in subtract
  ratio = num - den
<ipython-input-219-db07af29f152>:209: RuntimeWarning: invalid value encountered in double_scalars
  ratio = num - den
<ipython-input-219-db07af29f152>:257: RuntimeWarning: invalid value encountered in double_scalars
  ratio = num - den


Iteration:  0
count 1
nitem:  0  with  0.4371162397109419
nitem:  1  with  -1.7421795238232058
nitem:  2  with  -1.304654071773013
nitem:  3  with  -2.262637449943894
oldgamma:  1      pr_sd_theta:  2.2000015713941834
Iteration:  10
count 2
nitem:  0  with  1.6128293866124275
nitem:  1  with  -1.0276045691568965
nitem:  2  with  -1.3506734173458765
nitem:  3  with  -0.8422314356728288
oldgamma:  1      pr_sd_theta:  5.028935065226805
Iteration:  20
count 3
nitem:  0  with  0.4093960858106128
nitem:  1  with  -2.148886925129096
nitem:  2  with  -0.5584097745759579
nitem:  3  with  -0.7100183999680388
oldgamma:  1      pr_sd_theta:  5.998000346915622
Iteration:  30
count 4
nitem:  0  with  0.4523679954773709
nitem:  1  with  -2.3047036366116243
nitem:  2  with  -2.9024035725653348
nitem:  3  with  -0.7405812081982937
oldgamma:  1      pr_sd_theta:  7.663005879532199
Iteration:  40
count 5
nitem:  0  with  0.5102541230586677
nitem:  1  with  -2.175875165982859
nitem:  2  with  -3.95022114

SyntaxError: 'return' outside function (<ipython-input-219-db07af29f152>, line 347)

In [222]:
samp_beta
samp_theta
samp_z
samp_w
samp_gamma
samp_sd_theta
samp_sd
samp_mle
accept_beta
accept_theta
accept_z
accept_w

array([inf, inf, inf, inf, inf])

In [131]:
count

0

In [117]:
from math import *

import numpy as np
import pandas as pd

import scipy

from math import *
import numpy as np
import scipy.stats

from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale





In [159]:
def onepl_lsrm_cont_missing(data,

                            ndim,
                            niter,
                            nburn,
                            nthin,
                            nprint,

                            jump_beta,
                            jump_theta,
                            jump_gamma,
                            jump_z,
                            jump_w,

                            pr_mean_beta,
                            pr_sd_beta,
                            pr_a_th_sigma,
                            pr_b_th_sigma,
                            pr_mean_theta,

                            pr_a_sigma,
                            pr_b_sigma,
                            pr_mean_gamma,
                            pr_sd_gamma,

                            missing):
    ##############################################################################

    def update_function(beta, theta, gamma):
        return (- pow((data[k, i] - beta[i] - theta[k] + gamma * dist[k, i]), 2) /
                (2 * pow(pr_sd, 2))
                )

    def rejection_algorithm(ratio):
        if ratio > 0.0:
            accept = 1
        else:
            un = np.random.uniform(1)
            if np.log(un) < ratio:
                accept = 1
            else:
                accept = 0

        return accept

    ##############################################################################

    nsample = data.shape[0]
    nitem = data.shape[1]

    pr_mean_z = pr_mean_w = 0.0
    pr_sd_z = pr_sd_w = pr_sd = pr_sd_theta = 1.0

    oldbeta = np.random.uniform(size=nitem)
    oldtheta = np.random.uniform(size=nsample)
    oldz = np.random.uniform(size=(nsample, ndim))
    oldw = np.random.uniform(size=(nitem, ndim))

    # oldbeta = np.random.uniform(nitem)
    # oldtheta = np.random.uniform(nsample)
    # oldz = np.random.rand(nsample, ndim)
    # oldw = np.random.rand(nitem, ndim)

    newbeta = oldbeta = oldbeta * 4.0 - 2.0
    newtheta = oldtheta = oldtheta * 4.0 - 2.0
    newz = oldz = oldz * 2.0 - 1.0
    neww = oldw = oldw * 2.0 - 1.0

    ##############################################################################

    oldgamma = newgamma = 1  # gamma1 = log(gamma)

    samp_beta = np.zeros(shape=((niter - nburn) // nthin, nitem))
    samp_theta = np.zeros(shape=((niter - nburn) // nthin, nsample))

    samp_z = np.zeros(shape=((niter - nburn) // nthin, nsample, ndim))
    samp_w = np.zeros(shape=((niter - nburn) // nthin, nitem, ndim))

    samp_sd_theta = np.zeros(shape=(niter - nburn) // nthin, )
    samp_sd = np.zeros(shape=(niter - nburn) // nthin, )
    samp_mle = np.zeros(shape=(niter - nburn) // nthin, )
    samp_gamma = np.zeros(shape=(niter - nburn) // nthin, )

    accept_beta = np.zeros(shape=nitem, )
    accept_theta = np.zeros(shape=nsample, )
    accept_z = np.zeros(shape=nsample, )
    accept_w = np.zeros(shape=nitem, )

    accept_gamma = 0
    accept = 0
    count = 0

    dist = np.zeros(shape=(nsample, nitem), )

    old_dist_k = np.zeros(nitem, )
    new_dist_k = np.zeros(nitem, )
    old_dist_i = np.zeros(nsample, )
    new_dist_i = np.zeros(nsample, )

    ##############################################################################

    for iter in range(niter):
        # dist(j,i) is distance of z_j and w_i

        dist = np.where(True, 0, dist)  # 매 이터레이션마다 거리 매트릭스를 0으로 리셋

        for i in range(nitem):
            for k in range(nsample):
                dist_temp = 0.0
                for j in range(ndim):
                    dist_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                    dist[k, i] = sqrt(dist_temp)

        # beta update
        for i in range(nitem):
            # TODO 컬럼부터 갱신하는 이유가 있는건가?

            newbeta[i] = np.random.normal(oldbeta[i], jump_beta, 1)
            old_like_beta = new_like_beta = 0.0

            for k in range(nsample):
                if data[k, i] != missing:
                    new_like_beta += update_function(newbeta, oldtheta, oldgamma)
                    old_like_beta += update_function(oldbeta, oldtheta, oldgamma)

            num = (new_like_beta +
                   scipy.stats.norm.logpdf(newbeta[i], pr_mean_beta, pr_sd_beta))
            den = (old_like_beta +
                   scipy.stats.norm.logpdf(oldbeta[i], pr_mean_beta, pr_sd_beta))
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                oldbeta[i] = newbeta[i]
                accept_beta[i] += 1.0 / (niter * 1.0)

            else:
                newbeta[i] = oldbeta[i]

        # theta update
        for k in range(nsample):
            newtheta[k] = np.random.normal(oldtheta[k], jump_theta, 1)
            old_like_theta = new_like_theta = 0.0

            for i in range(nitem):
                if data[k, i] != missing:
                    new_like_theta += update_function(oldbeta, newtheta, oldgamma)
                    old_like_theta += update_function(oldbeta, oldtheta, oldgamma)

            num = (new_like_theta +
                   scipy.stats.norm.logpdf(newtheta[k], pr_mean_theta, pr_sd_theta))
            den = (old_like_theta +
                   scipy.stats.norm.logpdf(oldtheta[k], pr_mean_theta, pr_sd_theta))
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                oldtheta[k] = newtheta[k]
                accept_theta[k] += 1.0 / (niter * 1.0)
            else:
                newtheta[k] = oldtheta[k]

        # gamma(log(gamma)) update

        newgamma = np.random.lognormal(log(oldgamma), jump_gamma, 1)
        old_like_gamma = new_like_gamma = 0.0

        for k in range(nsample):
            for i in range(nitem):
                if data[k, i] != missing:
                    new_like_gamma += update_function(oldbeta, newtheta, newgamma)
                    old_like_gamma += update_function(oldbeta, newtheta, oldgamma)

        num = (new_like_gamma +
               scipy.stats.lognorm.logpdf(oldgamma, s=jump_gamma, loc=0, scale=exp(log(newgamma))) +
               scipy.stats.lognorm.logpdf(newgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
               )
        den = (old_like_gamma +
               scipy.stats.lognorm.logpdf(newgamma, s=jump_gamma, loc=0, scale=exp(log(oldgamma))) +
               scipy.stats.lognorm.logpdf(oldgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
               )
        ratio = num - den

        accept = rejection_algorithm(ratio)

        if accept == 1:
            oldgamma = newgamma
            accept_gamma += 1.0 / (niter * 1.0)
        else:
            newgamma = oldgamma

        # zj update

        for k in range(nsample):
            for j in range(ndim):
                newz[k, j] = np.random.normal(oldz[k, j], jump_z, 1)
            old_like_z = new_like_z = 0.0

            # calculate distance of oldw and newz

            for i in range(nitem):
                dist_old_temp = dist_new_temp = 0.0
                for j in range(ndim):
                    dist_new_temp += pow((newz[k, j] - oldw[i, j]), 2.0)
                    dist_old_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                new_dist_k[i] = sqrt(dist_new_temp)
                old_dist_k[i] = sqrt(dist_old_temp)

            # calculate likelihood

            for i in range(nitem):
                if data[k, i] != missing:
                    new_like_z += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * new_dist_k[i]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )
                    old_like_z += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * old_dist_k[i]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )

            num = den = 0.0

            for j in range(ndim):
                num += scipy.stats.norm.logpdf(newz[k, j], pr_mean_z, pr_sd_z)
                den += scipy.stats.norm.logpdf(oldz[k, j], pr_mean_z, pr_sd_z)

            # Rprintf("%.3f %.3f %.3f %.3f\n", num, den, new_like_z, old_like_z)
            # arma::dvec newzz = dmvnorm(newz.cols(2*j,2*j+1),pr_mean_z,pr_cov_z,TRUE)
            # arma::dvec oldzz = dmvnorm(oldz.cols(2*j,2*j+1),pr_mean_z,pr_cov_z,TRUE)

            num += new_like_z
            den += old_like_z
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                for j in range(ndim):
                    oldz[k, j] = newz[k, j]
                accept_z[k] += 1.0 / (niter * 1.0)

            else:
                for j in range(ndim):
                    newz[k, j] = oldz[k, j]

        #  wi update
        for i in range(nitem):
            for j in range(ndim):
                neww[i, j] = np.random.normal(oldw[i, j], jump_w, 1)
            old_like_w = new_like_w = 0.0

            # calculate distance of neww and oldz

            for k in range(nsample):
                dist_old_temp = dist_new_temp = 0.0
                for j in range(ndim):
                    dist_new_temp += pow((oldz[k, j] - neww[i, j]), 2.0)  # TODO: Why Old - New?
                    dist_old_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                new_dist_i[k] = sqrt(dist_new_temp)
                old_dist_i[k] = sqrt(dist_old_temp)

            # calculate likelihood

            for k in range(nsample):
                if data[k, i] != missing:
                    new_like_w += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * new_dist_i[k]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )
                    old_like_w += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * old_dist_i[k]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )

            num = den = 0.0

            for j in range(ndim):
                num += scipy.stats.norm.logpdf(neww[i, j], pr_mean_w, pr_sd_w)
                den += scipy.stats.norm.logpdf(oldw[i, j], pr_mean_w, pr_sd_w)

            num += new_like_w
            den += old_like_w
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                for j in range(ndim):
                    oldw[i, j] = neww[i, j]
                accept_w[i] += 1.0 / (niter * 1.0)
            else:
                for j in range(ndim):
                    neww[i, j] = oldw[i, j]

        # sigma_theta update with gibbs

        post_a_th_sigma = pr_a_th_sigma * 2 + nsample
        post_b_th_sigma = pr_b_th_sigma

        for j in range(nsample):  # TODO: 여기 j인데 nsample인거 맞음?
            post_b_th_sigma += pow((oldtheta[j] - pr_mean_theta), 2.0)
        pr_sd_theta = sqrt(2 * post_b_th_sigma * (1.0 / np.random.chisquare(post_a_th_sigma)))

        # dist(j,i) is distance of z_j and w_i

        dist = np.where(True, 0, dist)

        for i in range(nitem):
            for k in range(nsample):
                dist_temp = 0.0
                for j in range(ndim):
                    dist_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                dist[k, i] = sqrt(dist_temp)

        # sigma update with gibbs

        post_a_sigma = pr_a_sigma * 2 + nsample * nitem
        post_b_sigma = pr_b_sigma

        for j in range(nsample):  # TODO: 여기 j인데 nsample인거 맞음?
            for i in range(nitem):
                post_b_sigma += pow((data[j, i] - oldbeta[i] - oldtheta[j] + oldgamma * dist[j, i]), 2.0) / 2

        pr_sd = sqrt(2 * post_b_sigma * (1.0 / np.random.chisquare(post_a_sigma)))

        # burn, thin

        if iter >= nburn and iter % nthin == 0:
            for i in range(nitem):
                samp_beta[count, i] = oldbeta[i]
            for k in range(nsample):
                samp_theta[count, k] = oldtheta[k]
            for i in range(nitem):
                for j in range(ndim):
                    samp_w[count, i, j] = oldw[i, j]
            for k in range(nsample):
                for j in range(ndim):
                    samp_z[count, k, j] = oldz[k, j]

            samp_gamma[count] = oldgamma
            samp_sd_theta[count] = pr_sd_theta
            samp_sd[count] = pr_sd

            mle = 0.0

            for i in range(nitem):
                mle += scipy.stats.norm.logpdf(oldbeta[i], pr_mean_beta, pr_sd_beta)
            for k in range(nsample):
                mle += scipy.stats.norm.logpdf(oldtheta[k], pr_mean_theta, pr_sd_theta)
            for i in range(nitem):
                for j in range(ndim):
                    mle += scipy.stats.norm.logpdf(oldw[i, j], pr_mean_w, pr_sd_w)
            for k in range(nsample):
                for j in range(ndim):
                    mle += scipy.stats.norm.logpdf(oldz[k, j], pr_mean_z, pr_sd_z)
            for k in range(nsample):
                for i in range(nitem):
                    mle += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * dist[k, i]), 2)
                            / (2 * pow(pr_sd, 2)))

            mle += scipy.stats.lognorm.logpdf(oldgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
            samp_mle[count] = mle

            count += 1

        if iter % nprint == 0:
            
            print("Iteration: ", iter)
            print("count", count)
            for i in range(nitem):
                print("nitem: ", i, " with ", oldbeta[i])
            print("oldgamma: ", oldgamma, "     pr_sd_theta: ", pr_sd_theta)

    return {"beta": samp_beta,
            "theta": samp_theta,
            "z": samp_z,
            "w": samp_w,
            "gamma": samp_gamma,
            "sigma_theta": samp_sd_theta,
            "sigma": samp_sd,
            "map": samp_mle,
            "accept_beta": accept_beta,
            "accept_theta": accept_theta,
            "accept_z": accept_z,
            "accept_w": accept_w,
            "accept_gamma": accept_gamma}


In [203]:
ndim = 2 #차원?
niter = 50
nburn = 0
nthin = 10 #thining lapse
nprint = 10

jump_beta = 0.3
jump_theta = 1.0
jump_w = 0.06
jump_z = 0.50
jump_gamma = 0.01

pr_mean_beta = 0
pr_sd_beta = 1
pr_mean_theta = 0
pr_sd_theta = 1
pr_mean_gamma = 0.0
pr_sd_gamma = 1.0
pr_a_sigma = 0.001
pr_b_sigma = 0.001
pr_a_th_sigma = 0.001
pr_b_th_sigma = 0.001

In [206]:
output["map"]

array([nan, nan, nan, nan, nan])

In [205]:
output = onepl_lsrm_cont_missing(logit_x,
                                 ndim, niter, nburn, nthin, nprint,
                                 jump_beta, jump_theta, jump_gamma, jump_z, jump_w,
                                 pr_mean_beta, pr_sd_beta, pr_a_th_sigma, pr_b_th_sigma, pr_mean_theta,
                                 pr_a_sigma, pr_b_sigma, pr_mean_gamma, pr_sd_gamma,
                                 99)

<ipython-input-202-9e0e1f85ea6f>:130: RuntimeWarning: invalid value encountered in double_scalars
  ratio = num - den
<ipython-input-202-9e0e1f85ea6f>:156: RuntimeWarning: invalid value encountered in double_scalars
  ratio = num - den
<ipython-input-202-9e0e1f85ea6f>:185: RuntimeWarning: invalid value encountered in subtract
  ratio = num - den
<ipython-input-202-9e0e1f85ea6f>:235: RuntimeWarning: invalid value encountered in double_scalars
  ratio = num - den
<ipython-input-202-9e0e1f85ea6f>:283: RuntimeWarning: invalid value encountered in double_scalars
  ratio = num - den


Iteration:  0
count 1
nitem:  0  with  0.21024175908722068
nitem:  1  with  0.5375555482163185
nitem:  2  with  1.7722642859101656
nitem:  3  with  -1.4539945844630005
oldgamma:  1      pr_sd_theta:  2.12076425826342
Iteration:  10
count 2
nitem:  0  with  -0.43245876246675397
nitem:  1  with  1.1837119884417662
nitem:  2  with  2.8393768287369943
nitem:  3  with  -0.4066235007350091
oldgamma:  1      pr_sd_theta:  5.041881457776828
Iteration:  20
count 3
nitem:  0  with  -0.0006913948871042075
nitem:  1  with  1.329378813781376
nitem:  2  with  3.4941805504506935
nitem:  3  with  -0.6547614209291794
oldgamma:  1      pr_sd_theta:  6.742976009273222
Iteration:  30
count 4
nitem:  0  with  0.6074624867233792
nitem:  1  with  0.8726983094641195
nitem:  2  with  3.5463431916724186
nitem:  3  with  -1.857962801891612
oldgamma:  1      pr_sd_theta:  8.911316033326958
Iteration:  40
count 5
nitem:  0  with  0.18422947223644554
nitem:  1  with  1.0850438957472408
nitem:  2  with  4.3045033535

----

In [100]:
x1 = datasets.load_iris()["target"]
x2 = datasets.load_iris()["data"]

data1 = pd.concat([pd.DataFrame(x1), pd.DataFrame(x2)], axis=1)

data = np.array(data1)

data = np.array(data1)

data_word = data[:, 0] # 데이터 인풋. 인풋된 데이터는 data. input 되는 것은 논문 더미들.
data = np.delete(data, 0, 1) # 0번째 column은 data_word. 뽑아내고 나머지 데이타 보존.

data = np.nan_to_num(data, copy=True, nan=99) # na인 cell은 값을 99로 설정.

data_m = data # 데이터 타입 매트릭스로 바뀐 데이터. 파이썬에선 pd.dataframe 사용

from sklearn.preprocessing import MinMaxScaler

data_m = MinMaxScaler().fit_transform(data_m)

logit_x = np.log(data_m / (1 - data_m)) # 매트릭스 데이터를 logit化


# <editor-fold desc="covid_20 parameters">

ndim = 2 #차원?
niter = 2000
nburn = 0
nthin = 5 #thining lapse
nprint = 1000

"""
모든 topic의 분포는 theta ~ 디리클레 알파를 따름

biterm 총체 B에서 biterm b를 뽑으면, 이 biterm이 어느 topic z에 속할지는 z ~ 다항분포 세타

topic z의 topic-word 분포는 phi_z ~ 디리클레 베타를 따름
topic = z, word = w. z가 정해졌을 때 이로부터 각 단어가 가지는 확률을 모아서 set으로 한것이 phi_z.

골라진 topic에 대응하는 topic-word 분포로부터 단어 2개가 골라질 확률은 w_i, w_j ~ 다항(phi_z)를 따름
"""

jump_beta = 0.3
jump_theta = 1.0
jump_w = 0.06
jump_z = 0.50
jump_gamma = 0.01

pr_mean_beta = 0
pr_sd_beta = 1
pr_mean_theta = 0
pr_sd_theta = 1
pr_mean_gamma = 0.0
pr_sd_gamma = 1.0
pr_a_sigma = 0.001
pr_b_sigma = 0.001
pr_a_th_sigma = 0.001
pr_b_th_sigma = 0.001
# </editor-fold> #TODO 뭐지? 패러미터 스태티스틱?
#TODO 뭐지? 패러미터 스태티스틱?

In [143]:
num-den

0.0

In [138]:
# Set 99 as missing
"""
output = onepl_lsrm_cont_missing(data,
                                 ndim, niter, nburn, nthin, nprint,
                                 jump_beta, jump_theta, jump_gamma, jump_z, jump_w,
                                 pr_mean_beta, pr_sd_beta, pr_a_th_sigma, pr_b_th_sigma, pr_mean_theta,
                                 pr_a_sigma, pr_b_sigma, pr_mean_gamma, pr_sd_gamma,
                                 99)
"""

output = onepl_lsrm_cont_missing(logit_x,
                                 ndim, niter, nburn, nthin, nprint,
                                 jump_beta, jump_theta, jump_gamma, jump_z, jump_w,
                                 pr_mean_beta, pr_sd_beta, pr_a_th_sigma, pr_b_th_sigma, pr_mean_theta,
                                 pr_a_sigma, pr_b_sigma, pr_mean_gamma, pr_sd_gamma,
                                 99)

<ipython-input-137-020e0c0e9c6a>:130: RuntimeWarning: invalid value encountered in double_scalars
  ratio = num - den
<ipython-input-137-020e0c0e9c6a>:155: RuntimeWarning: invalid value encountered in double_scalars
  ratio = num - den
<ipython-input-137-020e0c0e9c6a>:184: RuntimeWarning: invalid value encountered in subtract
  ratio = num - den
<ipython-input-137-020e0c0e9c6a>:234: RuntimeWarning: invalid value encountered in double_scalars
  ratio = num - den
<ipython-input-137-020e0c0e9c6a>:282: RuntimeWarning: invalid value encountered in double_scalars
  ratio = num - den


Iteration:  0
count 1
nitem:  0  with  1.8620152940472952
nitem:  1  with  1.7135343004546215
nitem:  2  with  -0.928583526886758
nitem:  3  with  -2.4909841654941762
oldgamma:  1      pr_sd_theta:  2.157425225792545
Iteration:  100
count 21
nitem:  0  with  3.8593755993853707
nitem:  1  with  4.021705574638587
nitem:  2  with  1.4563587144486367
nitem:  3  with  -5.59671424349839
oldgamma:  1      pr_sd_theta:  16.50768601942056
Iteration:  200
count 41
nitem:  0  with  -1.1603885734927326
nitem:  1  with  4.231196366139966
nitem:  2  with  0.7527778666273348
nitem:  3  with  -6.77103984921703
oldgamma:  1      pr_sd_theta:  19.798168443823723
Iteration:  300
count 61
nitem:  0  with  -1.7941148890949754
nitem:  1  with  3.3316204153039854
nitem:  2  with  0.9765690471059602
nitem:  3  with  -10.479135224425596
oldgamma:  1      pr_sd_theta:  22.304793192823585
Iteration:  400
count 81
nitem:  0  with  2.3734707089639167
nitem:  1  with  0.7416839825144441
nitem:  2  with  -5.46935612

In [129]:
output["map"]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [109]:
"""
#output["accept_beta"].T
#output["accept_theta"]
### View(t(output$accept_theta))

#output["accept_w"].T
#output["accept_gamma"].T
### View(t(output$accept_z))

#output$accept_z
##Rcode procrustes matching
""" #TODO View Output Table?


def procrustes_mine(X, X_star):
    n = X.shape[0]
    m = X.shape[1]

    J = np.identity(n)

    C = X_star.transpose() @ J @ X
    svd_out = np.linalg.svd(C)
    R = svd_out[3] @ svd_out[1].transpose
    s = 1

    tt = np.zeros((m, 1))

    X.new = s * X @ R + tt.reshape(X.shape[0], X.shape[1])
    return X.new

# <editor-fold desc="MCMC Process">

In [110]:
nsample = data.shape[0]
nitem  = data.shape[1]

nmcmc = int((niter - nburn) / nthin)

max_address = min(output['map'].index(max(output['map'])))

w_star = output['w'].iloc[max_address, :, :]
z_star = output['z'].iloc[max_address, :, :]

w_proc = np.zeros((nmcmc, nitem, ndim), )
z_proc = np.zeros((nmcmc, nsample, ndim), )

for iter in range(nmcmc):
    z_iter = output['z'].iloc[iter,:,:]

    if iter != max_address:
        z_proc[iter, :, :] = procrustes_mine(z_iter, z_star)
        #z_proc[iter,:,:] = """procrustes(z_iter, z_star)$X.new""" #TODO ======= 210717 =======
    else: z_proc[iter,:,:] = z_iter

    w_iter = output['w'].iloc[iter,:,:]

    if iter != max_address:
        w_proc[iter,:,:] = procrustes_mine(w_iter, w_star) #TODO ======= 210717 =======
    else: w_proc[iter,:,:] = w_iter



w_est = np.empty((nitem, ndim,))

for i in range(nitem):
    for j in range(ndim):
        w_est[i, j] = w_proc[:, i, j].mean


z_est = np.empty((nsample, ndim,))

for k in range(nsample):
    for j in range(ndim):
        z_est[k, j] = z_proc[:, k, j].mean


beta_est = output["beta"].mean
theta_est = output["theta"].mean

#beta_est = apply(output["beta"], 2, mean)
#theta_est = apply(output["theta"], 2, mean)

sigma_theta_est = output["sigma_theta"].mean
gamma_est = output["gamma"].mean

output_new = {"beta_estimate" : beta_est,
              "theta_estimate" : theta_est,
              "sigma_theta_estimate" : sigma_theta_est,
              "gamma_estimate" : gamma_est,
              "z_estimate" : z_est,
              "w_estimate" : w_est,
              "beta" : output["beta"],
              "theta" : output["theta"],
              "theta_sd" : output["sigma_theta"],
              "gamma" : output["gamma"],
              "z" : z_proc,
              "w" : w_proc,
              "accept_beta" : output["accept_beta"],
              "accept_theta" : output["accept_theta"],
              "accept_w" : output["accept_w"],
              "accept_z" : output["accept_z"],
              "accept_gamma" : output["accept_gamma"]
              }

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [116]:
#(output).keys()

(output)["map"]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [ ]:
"""
#t(output_new$accept_beta)
#t(output_new$accept_theta)
#t(output_new$accept_w)
#t(output_new$accept_z)

#output_new["accept_gamma"]

# save.image("full_continuous_60.RData")
# save.image("full_continuous_70.RData")
# save.image("full_continuous_80.RData")
# save.image("full_continuous_mean.RData")
"""

"""
save.image("90_result.RData")

dir
pdf(paste0(dir, "/trace_beta.pdf"))
for (i in 1:ncol(output_new$beta)) ts.plot(output_new$beta[1:nrow(output_new$beta), i], main = paste("beta", i))
dev.off()

pdf(paste0(dir, "/trace_theta.pdf"))
for (i in 1:ncol(output_new$theta)) ts.plot(output_new$theta[, i], main=paste("theta", i))
dev.off()

pdf(paste0(dir, "/trace_sigma_theta.pdf"))
ts.plot(output_new$theta_sd, main = "sigma_theta")
dev.off()

pdf(paste0(dir, "/trace_gamma.pdf"))
ts.plot(output_new$gamma, main = "gamma")
dev.off()
"""
# </editor-fold>





"""
pdf(paste0(dir, "/trace_z.pdf"))
for (k in 1:ncol(output_new$theta)) ts.plot(output_new$z[, k, 1], main=paste("z_1", k))
for (k in 1:ncol(output_new$theta)) ts.plot(output_new$z[, k, 2], main=paste("z_2", k))
dev.off()

pdf(paste0(dir, "/trace_w.pdf"))
for (i in 1:ncol(output_new$beta)) ts.plot(output_new$w[, i, 1], main=paste("w_1", i))
for (i in 1:ncol(output_new$beta)) ts.plot(output_new$w[, i, 2], main=paste("w_2", i))
dev.off()
""" #TODO Multiple Write multiple Time Series Plot in pdf

In [ ]:


"""

gg = ggplot() +
    geom_text(data=bnew, aes(x=coordinate_1, y=coordinate_2, label=id), col=2) +
    # geom_text(data = b, aes(x=coordinate_1, y = coordinate_2, label = topic_name),col=2) +  #topic name
    # geom_point(data = anew,aes(x=coordinate_1,y=coordinate_2),cex=1) +
    xlim(min(bnew$coordinate_1, anew$coordinate_1)-0.2, max(bnew$coordinate_1, anew$coordinate_1)+0.2) +
    ylim(min(bnew$coordinate_2, anew$coordinate_2)-0.2, max(bnew$coordinate_2, anew$coordinate_2)+0.2)
    # xlim (-0.8,0.8) + ylim(-.8,.8)
print(gg)

b = bnew
a = anew

dir





pdf(paste0(dir, "/plot_wz.pdf"))



gg = ggplot() +
    geom_text(data=b, aes(x=coordinate_1, y=coordinate_2, label=id), col=2) +
    # geom_text(data = b, aes(x=coordinate_1, y = coordinate_2, label = topic_name),col=2) +  #topic name
    geom_point(data=a, aes(x=coordinate_1, y=coordinate_2), cex=1) +
    xlim(min(b$coordinate_1, a$coordinate_1)-0.2, max(b$coordinate_1, a$coordinate_1)+0.2) +
    ylim(min(b$coordinate_2, a$coordinate_2)-0.2, max(b$coordinate_2, a$coordinate_2)+0.2)
    # xlim (-0.8,0.8) + ylim(-.8,.8)
print(gg)

gg = ggplot() +
    # geom_text(data = b, aes(x=coordinate_1, y = coordinate_2, label = id),col=2) +
    geom_text(data=b, aes(x=coordinate_1, y=coordinate_2, label=topic_name), col=2) +  # topic name
    geom_point(data=a, aes(x=coordinate_1, y=coordinate_2), cex=1) +
    xlim(min(b$coordinate_1, a$coordinate_1)-0.2, max(b$coordinate_1, a$coordinate_1)+0.2) +
    ylim(min(b$coordinate_2, a$coordinate_2)-0.2, max(b$coordinate_2, a$coordinate_2)+0.2)
print(gg)



dev.off()
"""




"""
pdf(paste0(dir, "/plot_wz_new.pdf"))



gg = ggplot() +
    geom_text(data=b, aes(x=coordinate_1, y=coordinate_2, label=id), col=2) +
    # geom_text(data = b, aes(x=coordinate_1, y = coordinate_2, label = topic_name),col=2) +  #topic name
    geom_point(data=a_new, aes(x=coordinate_1, y=coordinate_2), cex=1) +
    xlim(min(b$coordinate_1, a_new$coordinate_1)-0.2, max(b$coordinate_1, a_new$coordinate_1)+0.2) +
    ylim(min(b$coordinate_2, a_new$coordinate_2)-0.2, max(b$coordinate_2, a_new$coordinate_2)+0.2)
    # xlim (-0.8,0.8) + ylim(-.8,.8)
print(gg)

gg = ggplot() +
    # geom_text(data = b, aes(x=coordinate_1, y = coordinate_2, label = id),col=2) +
    geom_text(data=b, aes(x=coordinate_1, y=coordinate_2, label=topic_name), col=2) +  # topic name
    geom_point(data=a_new, aes(x=coordinate_1, y=coordinate_2), cex=1) +
    xlim(min(b$coordinate_1, a_new$coordinate_1)-0.2, max(b$coordinate_1, a_new$coordinate_1)+0.2) +
    ylim(min(b$coordinate_2, a_new$coordinate_2)-0.2, max(b$coordinate_2, a_new$coordinate_2)+0.2)
print(gg)



dev.off()
"""



"""
pdf(paste0(dir, "/Plot5_3dplot_beta.pdf"))



topic_plot = scatterplot3d(new[, -4], pch = 16, color = colors, angle = 50)
text(topic_plot$xyz.convert(new[, -4]+0.3), labels = new$topics)

topic_plot = scatterplot3d(new[, -4], pch = 16, color = colors, angle = 50)
text(topic_plot$xyz.convert(new[, -4]+0.3), labels = c(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
                                                       19, 20))


dev.off()
"""

# TODO =====================

"""
##plot of topic

pdf(paste0(dir, "/Plot1_topic_cluter.pdf")) 



ggg = ggplot() +
    geom_point(data=b, aes(x=coordinate_1, y=coordinate_2), col=0.5) +
    geom_text(data=b, aes(x=coordinate_1, y=coordinate_2, label=topic_name), col=as.factor(group), cex = 3) +
    # geom_point(data = a, aes(x=coordinate_1,y= coordinate_2), cex=1) +
    # geom_text(data = a[group_index,], aes(x=coordinate_1, y = coordinate_2+0.15, label = dbscan$cluster[group_index]),col=2) +
    scale_color_manual(values=c("black", brewer.pal(n=9, name='Set1'))) +
    # geom_point(data = a[!dbscan$isseed,],aes(coordinate_1, coordinate_2), shape=8, cex=1) +
    xlim(min(b$coordinate_1, a$coordinate_1)-0.2, max(b$coordinate_1, a$coordinate_1)+0.2) +
    ylim(min(b$coordinate_2, a$coordinate_2)-0.2, max(b$coordinate_2, a$coordinate_2)+0.2) +
    # xlim (-0.8,1.2) + ylim(-.8,.1) +
    theme_bw() + theme(legend.position = "None")

print(ggg)



dev.off()
"""



###############################

word_position = pd.concat([data_word, a.iloc[:, 0:1]], axis=1)


word_position["dist"] = (word_position["coordinate_1"]**2 + word_position["coordinate_2"]**2)**0.5
word_new = word_position.loc[word_position["dist"] > 1.4, :]


"""
##plot of topic

pdf(paste0(dir, "/topic_cluter_withwords.pdf"))


ggg = ggplot() +
    geom_point(data=b, aes(x=coordinate_1, y=coordinate_2), col=0.5) +
    geom_text(data=b, aes(x=coordinate_1, y=coordinate_2, label=topic_name), col=as.factor(group), cex = 3) +
    # geom_point(data = a, aes(x=coordinate_1,y= coordinate_2), cex=1) +
    # geom_text(data = a[group_index,], aes(x=coordinate_1, y = coordinate_2+0.15, label = dbscan$cluster[group_index]),col=2)+
    scale_color_manual(values=c("black", brewer.pal(n=9, name='Set1'))) +
    # geom_point(data = a[!dbscan$isseed,],aes(coordinate_1, coordinate_2), shape=8, cex=1) +
    geom_point(data=a_new, aes(x=coordinate_1, y=coordinate_2), cex=1) +
    geom_label_repel(data=word_new, aes(x=coordinate_1, y=coordinate_2, label=data_word)) +
    xlim(min(b$coordinate_1, a_new$coordinate_1)-0.2, max(b$coordinate_1, a_new$coordinate_1)+0.2) +
    ylim(min(b$coordinate_2, a_new$coordinate_2)-0.2, max(b$coordinate_2, a_new$coordinate_2)+0.2) +
    theme_bw() + theme(legend.position = "None")
print(ggg)



dev.off()
"""


quantile(word_position$dist, c(0.05, 0.2))
word_new = word_position[word_position["dist"] < 0.41,]

#word_new

"""
pdf(paste0(dir,"/Plot4_topic_cluter_centerwords.pdf"))

##plot of topic 
ggg = ggplot() +
  geom_point(data = b, aes(x=coordinate_1, y = coordinate_2),col=0.5) +
  geom_text(data = b, aes(x=coordinate_1, y = coordinate_2, label = colnames(data_set2)),col=as.factor(group), cex=3) +
  # geom_point(data = a, aes(x=coordinate_1,y= coordinate_2), cex=1) +
  # geom_text(data = a[group_index,], aes(x=coordinate_1, y = coordinate_2+0.15, label = dbscan$cluster[group_index]),col=2)+
  scale_color_manual(values = c("black", brewer.pal(n = 9, name = 'Set1'))) +
  # geom_point(data = a[!dbscan$isseed,],aes(coordinate_1, coordinate_2), shape=8, cex=1) +
  geom_point(data = a_new,aes(x=coordinate_1,y=coordinate_2),cex=1) + 
  geom_label_repel(data = word_new, aes(x=coordinate_1,y=coordinate_2, label = data_word))+
  xlim (min(b$coordinate_1,a_new$coordinate_1)-0.2,max(b$coordinate_1,a_new$coordinate_1)+0.2) + 
  ylim(min(b$coordinate_2,a_new$coordinate_2)-0.2,max(b$coordinate_2,a_new$coordinate_2)+0.2) +
  theme_bw() + theme(legend.position = "None") 
print(ggg)
dev.off()
"""

words = data_word
close_word_index = []

## cosine similarity

temp = pd.concat([b.iloc[:, 1:2], group], axis = 1)
#head(temp)

## center of topic group


#temp2 = data.frame(temp % > % group_by(group) % > % summarise(x=mean(coordinate_1), y=mean(coordinate_2)))

temp2 = temp.groupby("group", axis=1).agg({"coordinate_1":"mean", "coordinate_2":"mean"})
temp2.columns = ["x", "y"]




#head(temp2)

word_position = pd.concat([words, a[, 1:2]], axis = 1)
close_word_index = []
#head(word_position)

library(ggrepel)


"""
pdf(paste0(dir, "/Plot3_topic_close_words.pdf"))

for (i in 1:ncol(data_set2)){z
    ## here 10
    # i = 1
    close_word_index[[i]] = order(as.matrix(dist(rbind(b[i, 1:2], a[, 1:2])))[1, -1])[1: 25]
    ggg = ggplot() +
        geom_text(data=b, aes(x=coordinate_1, y=coordinate_2, label=seq(1: ncol(data_set2))), col =as.factor(group), cex = 3) +
        geom_point(data=word_position[close_word_index[[i]],], aes(x=coordinate_1, y=coordinate_2), cex=1) +
        geom_point(data=b[i,], aes(x=coordinate_1, y=coordinate_2), col='red', cex=7, shape=2, stroke=1) +
        scale_color_manual(values=c("black", brewer.pal(n=9, name='Set1'))) +
        xlim(min(b$coordinate_1, a$coordinate_1)-0.5, max(b$coordinate_1, a$coordinate_1)+0.5) +
        ylim(min(b$coordinate_2, a$coordinate_2)-0.5, max(b$coordinate_2, a$coordinate_2)+0.5) +
        labs(title=paste("Topic", i), x="", y="") +
        theme_bw() + theme(legend.position = "None")
    
    # ggg <- ggg+ geom_text_repel(data = word_position[close_word_index,], aes(x=X1, y = X2, label = covid_20_word), size=3)

    ggg1 = ggg + 
        geom_label_repel(data=word_position[close_word_index[[i]],],
                         aes(x=coordinate_1, y=coordinate_2, label=words),
                         # fontface = 'bold', color = 'white', box.padding = unit(0.5, "lines"),
                         point.padding = unit(0.5, "lines"),
                         segment.color = 'grey50'
                        )
    print(ggg1)
}
dev.off()
"""

##### Cluster_Group near words

def euc_dist(x1, x2): sqrt(sum((x1 - x2)**2))

#head(word_position)

"""
pdf(paste0(dir, "/Plot2_cluster_close_words.pdf"))



for (i in 1:nrow(temp2)){
    # i = 1
    ## here 10
    cluster_word_dist < -c()
    close_word_index < -c()
    
    for (k in c(1:nrow(output_new$z_estimate))){
        cluster_word_dist[k]=euc.dist(temp2[i, -1], output_new$z_estimate[k, ])
    }
    close_word_index < -order(as.matrix(cluster_word_dist))[1: 25]
    ggg = ggplot() +
        geom_text(data=b, aes(x=coordinate_1, y=coordinate_2, label=seq(1: ncol(data_set2))), col =as.factor(group), cex = 3) +
        geom_point(data=word_position[close_word_index,], aes(x=coordinate_1, y=coordinate_2), cex=1) +
        geom_text(data=temp2[, -1], aes(x=x, y=y, label=paste("Group", seq(1, nrow(temp2)), sep="")), col = "red", cex = 3) +
        # geom_point(data = b[i,], aes(x=coordinate_1,y= coordinate_2), col='red', cex=7, shape=2, stroke = 1) +
        scale_color_manual(values=c("black", brewer.pal(n=9, name='Set1'))) +
        labs(title=paste("Cluster", i), x="", y="") +
        theme_bw() + theme(legend.position = "None")

    print(ggg)
    ggg1 = ggg + geom_label_repel(data=word_position[close_word_index,], 
                                    aes(x=coordinate_1, y=coordinate_2, label=words),
                                    # fontface = 'bold', color = 'white',
                                    box.padding = unit(0.5, "lines"),
                                    point.padding = unit(0.5, "lines"),
                                    segment.color = 'grey50'
                                )
    print(ggg1)
}



dev.off()
"""

save.image("75_result.RData")

SyntaxError: invalid syntax (<ipython-input-98-732a16ee9ab6>, line 286)